In [13]:
from google.cloud import bigquery
import pyarrow

In [14]:
def implicit():
    from google.cloud import storage

    # If you don't specify credentials when constructing the client, the
    # client library will look for credentials in the environment.
    storage_client = storage.Client()

    # Make an authenticated API request
    buckets = list(storage_client.list_buckets())
    print(buckets)

The first step in the workflow is to create a Client object. As you'll soon see, this Client object will play a central role in retrieving information from BigQuery datasets.

In [15]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/lihanwei/Downloads/formidable-fort-342720-747e794ef426.json"

In [16]:
# Create a "Client" object
client = bigquery.Client()

We'll work with a dataset of posts on Hacker News, a website focusing on computer science and cybersecurity news.

In BigQuery, each dataset is contained in a corresponding project. In this case, our hacker_news dataset is contained in the bigquery-public-data project. To access the dataset,

We begin by constructing a reference to the dataset with the dataset() method.
Next, we use the get_dataset() method, along with the reference we just constructed, to fetch the dataset.

In [17]:
# Construct a reference to the "hacker_news" dataset
dataset_ref = client.dataset("hacker_news", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

Every dataset is just a collection of tables. You can think of a dataset as a spreadsheet file containing multiple tables, all composed of rows and columns.

We use the list_tables() method to list the tables in the dataset.

In [18]:
# List all the tables in the "hacker_news" dataset
tables = list(client.list_tables(dataset))

# Print names of all tables in the dataset (there are four!)
for table in tables:  
    print(table.table_id)

comments
full
full_201510
stories


Similar to how we fetched a dataset, we can fetch a table. In the code cell below, we fetch the full table in the hacker_news dataset.

In [19]:
# Construct a reference to the "full" table
table_ref = dataset_ref.table("full")

# API request - fetch the table
table = client.get_table(table_ref)

# Table schema
The structure of a table is called its schema. We need to understand a table's schema to effectively pull out the data we want.

In this example, we'll investigate the full table that we fetched above.

In [20]:
# Print information on all the columns in the "full" table in the "hacker_news" dataset
table.schema

[SchemaField('title', 'STRING', 'NULLABLE', 'Story title', (), ()),
 SchemaField('url', 'STRING', 'NULLABLE', 'Story url', (), ()),
 SchemaField('text', 'STRING', 'NULLABLE', 'Story or comment text', (), ()),
 SchemaField('dead', 'BOOLEAN', 'NULLABLE', 'Is dead?', (), ()),
 SchemaField('by', 'STRING', 'NULLABLE', "The username of the item's author.", (), ()),
 SchemaField('score', 'INTEGER', 'NULLABLE', 'Story score', (), ()),
 SchemaField('time', 'INTEGER', 'NULLABLE', 'Unix time', (), ()),
 SchemaField('timestamp', 'TIMESTAMP', 'NULLABLE', 'Timestamp for the unix time', (), ()),
 SchemaField('type', 'STRING', 'NULLABLE', 'Type of details (comment, comment_ranking, poll, story, job, pollopt)', (), ()),
 SchemaField('id', 'INTEGER', 'NULLABLE', "The item's unique id.", (), ()),
 SchemaField('parent', 'INTEGER', 'NULLABLE', 'Parent comment ID', (), ()),
 SchemaField('descendants', 'INTEGER', 'NULLABLE', 'Number of story or poll descendants', (), ()),
 SchemaField('ranking', 'INTEGER', '

Each SchemaField tells us about a specific column (which we also refer to as a field). In order, the information is:

The name of the column
The field type (or datatype) in the column
The mode of the column ('NULLABLE' means that a column allows NULL values, and is the default)
A description of the data in that column
The first field has the SchemaField:

SchemaField('by', 'string', 'NULLABLE', "The username of the item's author.",())

This tells us:

the field (or column) is called by,
the data in this field is strings,
NULL values are allowed, and
it contains the usernames corresponding to each item's author.
We can use the list_rows() method to check just the first five lines of of the full table to make sure this is right. (Sometimes databases have outdated descriptions, so it's good to check.) This returns a BigQuery RowIterator object that can quickly be converted to a pandas DataFrame with the to_dataframe() method.

In [21]:
# Preview the first five lines of the "full" table
client.list_rows(table, max_results=5).to_dataframe()

,title,url,text,dead,by,score,time,timestamp,type,id,parent,descendants,ranking,deleted
0,None,None,+1 for Starwhal; we played it so much in schoo...,None,toraora,NaN,1572373754,2019-10-29 18:29:14+00:00,comment,21390509,21390248,NaN,NaN,None
1,None,None,Long ago someone already had the bright idea o...,None,soulofmischief,NaN,1572373752,2019-10-29 18:29:12+00:00,comment,21390508,21390324,NaN,NaN,None
2,None,None,Seems to me this should be verified as legit d...,None,alistproducer2,NaN,1475610309,2016-10-04 19:45:09+00:00,comment,12638720,12638449,NaN,NaN,None
3,None,None,Here (Oregon) was the first place to legalize ...,None,davidw,NaN,1475610322,2016-10-04 19:45:22+00:00,comment,12638721,12635494,NaN,NaN,None
4,None,None,Here&#x27;s a great &quot;wat&quot; talk for A...,None,evan121,NaN,1475610324,2016-10-04 19:45:24+00:00,comment,12638722,12638187,NaN,NaN,None


The list_rows() method will also let us look at just the information in a specific column. If we want to see the first five entries in the by column, for example, we can do that!

In [22]:
# Preview the first five entries in the "by" column of the "full" table
client.list_rows(table, selected_fields=table.schema[:1], max_results=5).to_dataframe()

,title
0,None
1,None
2,None
3,None
4,None


# Select, From & Where

First, set up everything to run queries and take a quick peek at the tables in database.

In [23]:
# Create a "client" object
client = bigquery.Client()

#Construct a reference to the "openaq" dataset
dataset_ref = client.dataset("openaq",project="bigquery-public-data")

#API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

In [24]:
# List all the tables in the "openaq" dataset
tables =list(client.list_tables(dataset))

# Print names of all tables in the dataset(there's only one!)
for table in tables:
    print(table.table_id)

global_air_quality
